In [ ]:
import pandas as pd
from nltk import word_tokenize, sent_tokenize
import os
from os.path import join
import stanza
import re
from collections import Counter
from langdetect import detect
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
ana = SentimentIntensityAnalyzer()
import requests
from langdetect import detect
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize,pos_tag
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bs4.element import Comment
# import credentials
# key = credentials.key
import xml.etree.ElementTree as ET
import string

punctuation_marks = [char for char in string.punctuation]

import string
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
for mark in string.punctuation + '“”’—‘':
    stopword_list.append(mark)

def remove_punctuation(words):
    new_list= []
    for w in words:
        if w.isalnum():
            new_list.append( w )
    return new_list

def sorted_by_value( dict , ascending = True ):
    if ascending: 
        return {k: v for k, v in sorted(dict.items(), key=lambda item: item[1])}
    else:
        return {k: v for k, v in reversed( sorted(dict.items(), key=lambda item: item[1]))}
    
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

api_key = 'QypW6JzQ4XfGViss1Ks4g'

In [ ]:
books = []

directory = 'Lists'
files = os.listdir(directory)
files = [file for file in files if not(re.search(r'^[.]',file))]

for list in files:

    path = os.path.join(directory,list)
    df = pd.read_csv(path,sep='\t')
    for i,row in df.iterrows():
        books.append( row['primary_isbn13'] )

books = set(books)
print(len(books))

In [ ]:
reviews = []

for isbn in books: 
    print(isbn)
    
    data = dict()
    
    api_call = f'https://www.goodreads.com/book/isbn?format=xml&key={api_key}&isbn={isbn}'
    print(api_call)
    
    headers = {'User-Agent': select_agent() }
    response = requests.get( api_call,headers=headers)
    xml_data = response.text
    
    tree = ET.ElementTree(ET.fromstring(xml_data))
    
    data['isbn'] = isbn
    data['average_rating'] = tree.find('book/average_rating').text
    data['ratings_count'] = tree.find('book/ratings_count').text
    data['text_reviews_count'] = tree.find('book/text_reviews_count').text
    
    path = join('Goodreads_reviews',f'reviews_{isbn}.txt')
    reviews_file = open(path,encoding='utf-8')

    sum_positive = 0
    sum_negative = 0
    sum_compound = 0
    
    count_lines = 0
    for line in reviews_file:
        count_lines += 1
        if re.search('^\d*\t',line):
            line = re.sub('^\d*\t','',line).strip().lower()
        words = word_tokenize(line)
        words = [word for word in words if word not in punctuation_marks]
        data['nr_tokens'] = data.get('nr_tokens',0) + len(words)
        sentences = sent_tokenize(line)
        data['nr_sentences'] = data.get('nr_sentences',0) + len(sentences)


        for word in words:
            sum_positive += ana.polarity_scores(word)['pos']
            sum_negative += ana.polarity_scores(word)['neg']
            sum_compound += ana.polarity_scores(word)['compound']

    data['downloaded_reviews'] = count_lines
    data['average_tokens'] = data['nr_tokens'] / data['downloaded_reviews']
    if data['nr_tokens']>0:
        data['positive'] = sum_positive/data['nr_tokens']
        data['negative'] = sum_negative/data['nr_tokens']
        data['compound'] = sum_compound/data['nr_tokens'] 
    reviews.append(data)

In [ ]:
df = pd.DataFrame(reviews)
df.to_csv('goodreads_statistics.csv')